In [11]:

all_symbols = list(u"`<>{}[]/*-+.!@#$%&*()_=+")
symbols = ['.','@']
fixed_pattern = "3010"
max_pass_length = 20
min_pass_length = 7
# -------------- password generator ------------------
# with open("pass.lst", 'wb') as f:
#     for i in range(min_pass_length,max_pass_length+1):
#         for cap in capitals:
#             for small in smalls:
#                 for sym in symbols:
# #             f.write(cap+)
#                     pass
    

In [12]:
from string import ascii_lowercase, ascii_uppercase, digits
from itertools import product, islice
from pprint import pprint

symbols = ".@"

key_map = {
    's_letters':ascii_lowercase,
    'u_letters':ascii_uppercase,
    'digits': digits,
    'symbols': symbols
}
all_pass = []

In [13]:
# Using list ---> 10000 loops, best of 3: 147 µs per loop
# Using set ---> 10000 loops, best of 3: 181 µs per loop

# After adding string conversion ---> 1000 loops, best of 3: 283 µs per loop

def get_password(l_bound=6, u_bound=20):
    first_key = 'u_letters'
    needed_keys = ['digits', 'symbols', 's_letters']
    with_capital_letter = False
    pwd = ""
    for n in range(l_bound, u_bound-3): # u_bound - 4 (for "3010") + 1 (range doesn't access last num)
        for tup in product(needed_keys,repeat=n): # to repreat product nth time
            for p in product(*(key_map[key] for key in tup)):
                pwd = "".join(p) + "3010"
                for extra in key_map[first_key]:
                    yield extra + pwd
                yield pwd
                
password = get_password()

In [14]:
islice(password, 26, 27).next()

'0000003010'

In [15]:
# 'Z0370373010'
password.next()

'A0000013010'

In [16]:
class PwdGenerator():

  def __init__(self, *args, **kwargs):
    self.l_bound = 6
    self.u_bound = 20
    self.key_map = {
        'small_letters':ascii_lowercase,
        'capital_letters':ascii_uppercase,
        'digits': digits,
        'symbols': u"`<>{}[]/*-+.!@#$%&*()_=+;:]'\""
    }
    # mapped using key_map
    self.prefix_key = None 
    self.suffix_key = None 
    
    self.prefix_key_len = 0
    self.suffix_key_len = 0
    
    self.prefix_fixed = ""
    self.suffix_fixed = ""
    
    self.suffix_part_confirmed = False
    self.prefix_part_confirmed = False
        
    self.suffix_key_confirmed = False
    self.prefix_key_confirmed = False
    
    self.needed_keys = []

    self.finished = False
    self.last_pwd = None
    
    self.generator_count = 0

  def init(self, data):
    """ set attributes due to your need """
    self.suffix_fixed = "3010"
    self.prefix_key = "capital_letters" # data.get('prefix_key', None)
    self.prefix_key_len = 1
    self.prefix_key_confirmed = False
    self.suffix_part_confirmed = True
    
    self.needed_keys = ['digits', 'symbols', 'small_letters']
    self.key_map['symbols'] = ".@"
    self.generator = self.generator_func()
  

  def generator_func(self):
    """ password generator """
    pwd = {
      'prefix' : self.prefix_fixed,
      'body': '',
      'suffix': self.suffix_fixed
    }
    length_part = len(self.prefix_fixed) * self.prefix_part_confirmed \
                + len(self.suffix_fixed) * self.suffix_part_confirmed \
                + self.prefix_key_len * self.prefix_key_confirmed \
                + self.suffix_key_len * self.suffix_key_confirmed
    for n in range(self.l_bound - length_part, self.u_bound + 1): # + 1 (range doesn't access last num)
        for tup in product(self.needed_keys,repeat=n): # to repreat product's input nth time
            for p in product(*(self.key_map[key] for key in tup)):
                pwd['body'] = "".join(p)
                if self.prefix_key in self.key_map.keys():
                  for extra in product(self.key_map[self.prefix_key], repeat=self.prefix_key_len):
                      pwd['prefix'] = self.prefix_fixed + "".join(extra)
                      yield pwd['prefix'] + pwd['body'] + pwd['suffix']
                
                if not self.prefix_part_confirmed:
                    yield pwd['body'] + pwd['suffix']
                
                if self.suffix_key in self.key_map.keys():
                  for extra in product(self.key_map[self.suffix_key], repeat=self.suffix_key_len):
                      pwd['suffix'] = "".join(extra) + self.suffix_fixed
                      yield pwd['prefix'] + pwd['body'] + pwd['suffix']

                if not self.suffix_part_confirmed:
                    yield pwd['prefix'] + pwd['body']
  
  def get(self):
    """ return next iteration in generator """
    try:
        pwd = self.generator.next()
        while self.last_pwd == pwd:
            pwd = self.generator.next()
        self.last_pwd = pwd
        self.generator_count += 1
        return pwd
    except StopIteration:
        return self.generator_end()
    finally:
        print("[{}] --> {}".format(self.generator_count, self.last_pwd))
  
  def skip(self, n):
    """ skip nth iterations in generator and return next iteration """
    previous_count = self.generator_count
    try:
        pwd = islice(self.generator, n, n+1).next() 
        self.last_pwd = pwd
        self.generator_count += n 
        return pwd
    except StopIteration as ex:
        print(ex)
        return self.generator_end()
    finally:
        print("[{0}] --> [{1}]: {2}".format(previous_count, self.generator_count, self.last_pwd))

  def reset(self):
     self.generator_count = 0
     self.generator = self.generator_func()   

  def generator_end(self):
    self.finished = True
    return "" # in this case empty_string is better than None

p = PwdGenerator()
p.init({})

In [17]:
p.skip(209)

[0] --> [209]: U073010


'U073010'

In [18]:
p.get()

[210] --> V073010


'V073010'

In [19]:
from random import randint
randint(1,5)

4

In [2]:
from threading import Thread
from time import sleep
def dd():
    print("Started")
    sleep(1)
    print("Finished")
    
t1 = Thread(name='test', target=dd)
t1.start()
print(t1.name)

Startedtest

Finished
